# Installing WebLLM in Pyodide with Micropip

This notebook shows how to install and use WebLLM packages in Pyodide using micropip.

In [ ]:
# Install WebLLM-related packages via micropip
import micropip

# Install JavaScript interop helpers
await micropip.install(['js', 'pyodide-js'])

print("Packages installed successfully!")

In [ ]:
# Load WebLLM from CDN
from pyodide import js
import asyncio

# Dynamically load WebLLM
js.eval("""
// Create a promise to load WebLLM
window.loadWebLLM = new Promise((resolve, reject) => {
    const script = document.createElement('script');
    script.src = 'https://cdn.jsdelivr.net/npm/@mlc-ai/web-llm@0.2.46/lib/index.min.js';
    script.onload = () => {
        console.log('WebLLM loaded');
        window.WebLLM = window.tvmjs.webllm;
        resolve(window.WebLLM);
    };
    script.onerror = reject;
    document.head.appendChild(script);
});
""")

# Wait for WebLLM to load
print("Loading WebLLM...")
await js.eval("window.loadWebLLM")
print("WebLLM loaded successfully!")

In [ ]:
# Create a Python wrapper for WebLLM
class PyodideWebLLM:
    def __init__(self):
        self.engine = None
        self.ready = False
    
    async def load_model(self, model_id="Llama-3.2-1B-Instruct-q4f16_1-MLC"):
        """Load a WebLLM model"""
        try:
            print(f"Loading model: {model_id}")
            self.engine = await js.WebLLM.CreateMLCEngine(model_id)
            self.ready = True
            print("Model loaded successfully!")
        except Exception as e:
            print(f"Error loading model: {e}")
            return False
        return True
    
    async def chat(self, message, max_tokens=512, temperature=0.7):
        """Chat with the model"""
        if not self.ready:
            return "Model not loaded. Call load_model() first."
        
        try:
            # Prepare chat messages
            messages = [{"role": "user", "content": message}]
            
            # Generate response
            response = await self.engine.chat.completions.create(
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature
            )
            
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {e}"
    
    async def complete(self, prompt, max_tokens=512, temperature=0.7):
        """Complete text"""
        if not self.ready:
            return "Model not loaded. Call load_model() first."
        
        try:
            response = await self.engine.completions.create(
                prompt=prompt,
                max_tokens=max_tokens,
                temperature=temperature
            )
            return response.choices[0].text
        except Exception as e:
            return f"Error: {e}"

# Create WebLLM instance
llm = PyodideWebLLM()
print("WebLLM wrapper created!")

In [ ]:
# Load the model (this may take a few minutes for first download)
await llm.load_model()
print("Ready to chat!")

In [ ]:
# Chat example
response = await llm.chat("Explain what Pyodide is in simple terms")
print("Response:")
print(response)

In [ ]:
# Text completion example
response = await llm.complete("The advantages of running Python in the browser are:")
print("Completion:")
print(response)

In [ ]:
# Interactive chat function
async def interactive_chat():
    """Simple interactive chat with WebLLM"""
    print("🤖 WebLLM Chat - Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        try:
            # Get user input (in Pyodide, we'll simulate this)
            user_input = input("You: ")
            
            if user_input.lower() in ['quit', 'exit']:
                print("Goodbye!")
                break
            
            if user_input.strip():
                response = await llm.chat(user_input)
                print(f"🤖: {response}")
                print()
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")

# Note: input() doesn't work in JupyterLite, so we'll demo with predefined messages
test_messages = [
    "Hello! What can you do?",
    "Tell me about machine learning",
    "How does WebLLM work?"
]

print("🤖 Demo Chat Session")
print("-" * 30)

for msg in test_messages:
    print(f"You: {msg}")
    response = await llm.chat(msg)
    print(f"🤖: {response}")
    print()